In [2]:
from models import models
models

{'multilingual': 'FacebookAI/xlm-roberta-base',
 'arq': 'CAMeL-Lab/bert-base-arabic-camelbert-da',
 'amh': 'Davlan/xlm-roberta-base-finetuned-amharic',
 'eng': 'FacebookAI/roberta-base',
 'hau': 'Davlan/xlm-roberta-base-finetuned-hausa',
 'kin': 'Davlan/xlm-roberta-base-finetuned-kinyarwanda',
 'mar': 'l3cube-pune/marathi-roberta',
 'ary': 'CAMeL-Lab/bert-base-arabic-camelbert-da',
 'esp': 'PlanTL-GOB-ES/roberta-base-bne',
 'tel': 'l3cube-pune/telugu-bert'}

In [3]:
from util import get_langs
langs = get_langs()
langs

['amh', 'arq', 'ary', 'eng', 'esp', 'hau', 'kin', 'mar', 'tel']

In [4]:
from pair_encoder.evaluation import CorrelationEvaluator, get_correlation
from pair_encoder.model import PairEncoder

import pandas as pd

from util import get_data, get_pairs, eval_and_submit

def eval_lang(model, lang, save_name, model_name="default"):
    data = {
        "dev": get_pairs(get_data(lang=lang, train=False)),
        "test": get_pairs(get_data(lang=lang, test=True))
    }
    correlations = {
        "dev": get_correlation(test=data["dev"], pair_encoder=model),
        "test": get_correlation(test=data["test"], pair_encoder=model)
    }
    eval_and_submit(
        pair_encoder=model,
        lang=lang,
        model_name=model_name,
        timestamp=f"{lang}-test-{save_name}",
        evaluation_phase=True
    )
    return correlations


In [5]:
from pair_encoder import train_encoder


for K in [1, 2, 3, 4, 5]:
    print(K)
    baseline_train_lang = {}
    sim_model = "intfloat/multilingual-e5-base"
    for lang in langs:
        print(f"Training on {lang}...")
        lang_train = get_data(lang=lang, train=True)
        lang_dev = get_data(lang=lang, train=False)

        train_pairs = get_pairs(lang_train)
        eval_pairs = get_pairs(lang_dev)
        evaluator = CorrelationEvaluator.load(eval_pairs)
        
        model_name = models[lang]
        print(f"Using model {model_name} for {lang}...")

        encoder, _ = train_encoder(
            train_samples=train_pairs,
            upscaling_samples=None,
            evaluator=evaluator,
            timestamp=f"{model_name}-train-{lang}",
            model_name=model_name,
            similarity_model=sim_model,
            batch_size=32,
            learning_rate=2e-5,
            max_grad_norm=1.0,
            epochs=5,
            eval_steps=0,
            max_length=200,
            k=K,
            weak_training_epochs=2,  # used if k > 0
            seed=42,
            save_to=None,
            verbose=True,
            device="cuda"
        )
        
        baseline_train_lang[lang] = eval_lang(encoder, lang, save_name=f"baseline-train-{K}-{lang}")
    df_baseline_lang = pd.DataFrame(baseline_train_lang).T
    df_prefix = f"custom-lang-k{K}"
    df_baseline_lang.columns = [f"{df_prefix}_{c}" for c in df_baseline_lang.columns]

    final_df = df_baseline_lang.T
    final_df = final_df * 100
    final_df = final_df.applymap(lambda x: round(x, 2))
    final_df
    columns = [
        "arq", "amh", "eng", "hau", "kin", "mar", "ary", "esp", "tel"
    ]
    final_df = final_df[columns]
    print(final_df.to_latex(float_format="%.2f"))

1
Training on amh...
Using model Davlan/xlm-roberta-base-finetuned-amharic for amh...


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at Davlan/xlm-roberta-base-finetuned-amharic and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training...:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/31 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/31 [00:00<?, ?it/s]

...

Epoch 1:   0%|          | 0/392 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/392 [00:00<?, ?it/s]

Epoch 3:   0%|          | 0/392 [00:00<?, ?it/s]

Epoch 4:   0%|          | 0/392 [00:00<?, ?it/s]

KeyboardInterrupt: 